<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab: Cleaning Rock Song Data

---


In [381]:
import pandas as pd
import numpy as np 
import seaborn as sns

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [382]:
rockfile = "../../../../resource-datasets/rock_songs/rock.csv"

In [383]:
# Load the data.
df=pd.read_csv(rockfile)

In [484]:
# Look at the information regarding its columns.
df.head()

,song,artist,release,song+artist,first,year,playcount,f*g,higher_than_1970
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82,True
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0,False
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85,True
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18,True
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1,True


### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [385]:
# Change the column names when loading the '.csv':
df=pd.read_csv("../../../../resource-datasets/rock_songs/rock.csv",names=['song','artist','released',
                                                                          'song+artist','first','year',
                                                                          'playcount','fg'],skiprows=1)
df.head()

,song,artist,released,song+artist,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [386]:
# Change the column names using the `.rename()` function.
new_dict={'song':'songs','artist':'arti','released':'release',
                   'song+artist':'songs+arti','first':'first?','year':'year?','playcount':'playc','fg':'f*g'}
df.rename(columns=new_dict,inplace=True)
df.head()

,songs,arti,release,songs+arti,first?,year?,playc,f*g
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [387]:
# Replace the column names by reassigning the `.columns` attribute.
df.columns=['song','artist','release','song+artist','first','year','playcount','f*g']
df.head()

,song,artist,release,song+artist,first,year,playcount,f*g
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [388]:
# Show records where df['release'] is null
df[df['release'].isnull()].head()

,song,artist,release,song+artist,first,year,playcount,f*g
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0


### 4. Update slices of your DataFrame based on mask selection/slices.

In many scenarios, we want to upate values in our DataFrame according to criteria. Let's say we wanted to set all of the null values in `release` to 0.

With newer versions of `pandas`, in order to manipulate data in the original DataFrame, we have to use `.loc` while performing reassignment using a mask and an index.

For example, the following won't always work:
```python
df[row_mask]['column_name'] = new_value
```

The best way to accomplish the same task is:
```python
df.loc[row_mask, 'column_name'] = new_value
```

For multiple column assignment, you would use:
```python
df.loc[row_mask, ['col_1', 'col_2', 'col_3']] = new_value
```

#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [389]:
# Replace release nulls with 0
df.loc[df[df['release'].isnull()].index,'release']=0
df.head(15)

,song,artist,release,song+artist,first,year,playcount,f*g
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
5,Kryptonite,3 Doors Down,2000,Kryptonite by 3 Doors Down,1,1,13,13
6,Loser,3 Doors Down,2000,Loser by 3 Doors Down,1,1,1,1
7,When I'm Gone,3 Doors Down,2002,When I'm Gone by 3 Doors Down,1,1,6,6
8,What's Up?,4 Non Blondes,1992,What's Up? by 4 Non Blondes,1,1,3,3
9,Take On Me,a-ha,1985,Take On Me by a-ha,1,1,1,1


#### 4.B Verify that `release` contains no null values.

In [390]:
# A:
df[df['release'].isnull()]

,song,artist,release,song+artist,first,year,playcount,f*g


### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

In [391]:
# A:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
song           2230 non-null object
artist         2230 non-null object
release        2230 non-null object
song+artist    2230 non-null object
first          2230 non-null int64
year           2230 non-null int64
playcount      2230 non-null int64
f*g            2230 non-null int64
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [392]:
# A:
df[(df['release']!=np.int()) & (df['release']!=np.str())].head()

,song,artist,release,song+artist,first,year,playcount,f*g
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
5,Kryptonite,3 Doors Down,2000,Kryptonite by 3 Doors Down,1,1,13,13


#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [393]:
# A:
for i in range(len(df)):
    try:
        float(df['release'][i])
    except:       
        print(df.iloc[i])
        print('row number: ',i)

song                             Bullfrog Blues
artist                           Rory Gallagher
release                           SONGFACTS.COM
song+artist    Bullfrog Blues by Rory Gallagher
first                                         1
year                                          1
playcount                                     1
f*g                                           1
Name: 1504, dtype: object
row number:  1504


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [394]:
# A:
k=0
for i in range(len(df)):
    try:
        float(df['release'][i])
    except:       
        df.loc[i,'release']=0
        k=i
df.loc[:,'release'].astype('int64')
df.loc[k]

song                             Bullfrog Blues
artist                           Rory Gallagher
release                                       0
song+artist    Bullfrog Blues by Rory Gallagher
first                                         1
year                                          1
playcount                                     1
f*g                                           1
Name: 1504, dtype: object

### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [406]:
# A:
df['release'].describe()

count     2230
unique      56
top          0
freq       578
Name: release, dtype: int64

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

# A: yes many of it is zeros, with 578 zeros.

### 8. Make changes and investigate using custom functions with `.apply()`.

Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.

In [480]:
# A:
def date_higher_1970(x):
    return np.int_(x['release'])>1970

#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [482]:
# A:
df['higher_than_1970']=df.apply(date_higher_1970,axis=1)
df[['song','artist','higher_than_1970']]

,song,artist,higher_than_1970
0,Caught Up in You,.38 Special,True
1,Fantasy Girl,.38 Special,False
2,Hold On Loosely,.38 Special,True
3,Rockin' Into the Night,.38 Special,True
4,Art For Arts Sake,10cc,True
5,Kryptonite,3 Doors Down,True
6,Loser,3 Doors Down,True
7,When I'm Gone,3 Doors Down,True
8,What's Up?,4 Non Blondes,True
9,Take On Me,a-ha,True


You'll notice that there will be a final output Series of `None` values. The `.apply()` function, if a return value is not specified, will return a Series of `None` values (similar to how the default return for Python functions is `None` when a return statement is not specified).

### 9. Write a function that converts cells in a DataFrame to float and otherwise replaces them with `np.nan`.

If applied to our data, it would keep only the numeric information and otherwise input null values.

Recall that the try-except syntax in Python is a great way to try something and take another action if the initial step fails:

```python
try:
    Perform some action.
except:
   Perform some other action if the first failed with an error.
```

#### 9.A Write the function that takes a column and converts all of its values to float if possible and `np.nan` otherwise. The return value should be the converted Series.

In [491]:
# A:
def float_or_nan(x):
    try:
        return float(x)
    except:
        return np.nan

#### 9.B Try your function out on the rock song data and ensure the output is what you expected.


In [493]:
# A:
df.applymap(float_or_nan).head()

,song,artist,release,song+artist,first,year,playcount,f*g,higher_than_1970
0,NaN,NaN,1982.0,NaN,1.0,1.0,82.0,82.0,1.0
1,NaN,NaN,0.0,NaN,1.0,0.0,3.0,0.0,0.0
2,NaN,NaN,1981.0,NaN,1.0,1.0,85.0,85.0,1.0
3,NaN,NaN,1980.0,NaN,1.0,1.0,18.0,18.0,1.0
4,NaN,NaN,1975.0,NaN,1.0,1.0,1.0,1.0,1.0


#### 9.C Describe the new float-only DataFrame.

In [498]:
# A:
df.applymap(float_or_nan).describe()

,song,artist,release,song+artist,first,year,playcount,f*g,higher_than_1970
count,2.000000,0.0,2230.000000,0.0,2230.0,2230.000000,2230.000000,2230.000000,2230.000000
mean,1012.000000,NaN,1465.331390,NaN,1.0,0.741256,16.872646,15.048430,0.584753
std,1367.544515,NaN,867.196161,NaN,0.0,0.438043,25.302972,25.288366,0.492875
min,45.000000,NaN,0.000000,NaN,1.0,0.000000,0.000000,0.000000,0.000000
25%,528.500000,NaN,0.000000,NaN,1.0,0.000000,1.000000,0.000000,0.000000
50%,1012.000000,NaN,1973.000000,NaN,1.0,1.000000,4.000000,3.000000,1.000000
75%,1495.500000,NaN,1981.000000,NaN,1.0,1.000000,21.000000,18.000000,1.000000
max,1979.000000,NaN,2014.000000,NaN,1.0,1.000000,142.000000,142.000000,1.000000


### 10. What are the top 20 most popular songs by plays?

In [499]:
# A:
df.sort_values(by='playcount',ascending=False).head(20)

,song,artist,release,song+artist,first,year,playcount,f*g,higher_than_1970
49,Dream On,Aerosmith,1973,Dream On by Aerosmith,1,1,142,142,True
868,All Along the Watchtower,Jimi Hendrix,1968,All Along the Watchtower by Jimi Hendrix,1,1,141,141,False
65,Sweet Emotion,Aerosmith,1975,Sweet Emotion by Aerosmith,1,1,141,141,True
38,You Shook Me All Night Long,AC/DC,1980,You Shook Me All Night Long by AC/DC,1,1,138,138,True
267,More Than a Feeling,Boston,1976,More Than a Feeling by Boston,1,1,134,134,True
968,Carry On Wayward Son,Kansas,1976,Carry On Wayward Son by Kansas,1,1,134,134,True
269,Peace of Mind,Boston,1976,Peace of Mind by Boston,1,1,132,132,True
774,Crazy On You,Heart,1976,Crazy On You by Heart,1,1,125,125,True
2220,Legs,ZZ Top,1983,Legs by ZZ Top,1,1,121,121,True
2224,Sharp Dressed Man,ZZ Top,1983,Sharp Dressed Man by ZZ Top,1,1,120,120,True


### 11. Which years have the most plays?

In [515]:
# A:
df.sort_values(by='playcount',ascending=False)[['release','playcount']].head()

,release,playcount
49,1973,142
868,1968,141
65,1975,141
38,1980,138
267,1976,134


### 12. Which records don't have matching "Play Count" corresponding to "F*G"?

In [530]:
# A:
df[df['release'] == df['f*g']]

,song,artist,release,song+artist,first,year,playcount,f*g,higher_than_1970
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0,False
10,"Baby, Please Don't Go",AC/DC,0,"Baby, Please Don't Go by AC/DC",1,0,1,0,False
13,CAN'T STOP ROCK'N'ROLL,AC/DC,0,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0,False
16,Girls Got Rhythm,AC/DC,0,Girls Got Rhythm by AC/DC,1,0,24,0,False
24,Let's Get It Up,AC/DC,0,Let's Get It Up by AC/DC,1,0,4,0,False
25,Live Wire,AC/DC,0,Live Wire by AC/DC,1,0,2,0,False
26,Moneytalks,AC/DC,0,Moneytalks by AC/DC,1,0,20,0,False
29,Shoot To Thrill,AC/DC,0,Shoot To Thrill by AC/DC,1,0,45,0,False
31,Sin City,AC/DC,0,Sin City by AC/DC,1,0,1,0,False
35,What Do You Do For Money Honey,AC/DC,0,What Do You Do For Money Honey by AC/DC,1,0,2,0,False


### Bonus: Which artists have the most missing values between each of the variables? 

In [544]:
# A:
